In [35]:
import spacy
from spacy.tokens import DocBin
from spacy.training import Example
from spacy.util import Config
from spacy.training import Corpus
import random
from tqdm import tqdm
import pandas as pd 
import os
import subprocess

print("all libraries loaded")

all libraries loaded


In [3]:
print("loading spacy model...")
nlp_fr = spacy.load("fr_core_news_sm")
print("spaCy model 'fr_core_news_sm' charge avec succes")

loading spacy model...
spaCy model 'fr_core_news_sm' charge avec succes


In [5]:
texte_simple = '''Je voudrais aller de Ermont à Sannois.'''
doc = nlp_fr(texte_simple)
spacy.displacy.render(doc, style="ent", jupyter=True)

In [7]:
# Charger le fichier CSV
df = pd.read_csv('../../dataset/raw/initial_training_data.csv')
print("\nfichier csv charge avec succes\n")


fichier csv charge avec succes



In [9]:
# premieres lignes du dataset pour verifier le chargement
print(df.head())

                                            sentence  entity_start  \
0  Quels sites touristiques visiter en allant de ...            46   
1  Quels sites touristiques visiter en allant de ...            54   
2  Combien de temps faut-il pour aller de PARIS à...            39   
3  Combien de temps faut-il pour aller de PARIS à...            47   
4  Quels sont les moyens de transport les plus éc...            70   

   entity_end  entity_type  
0          51    DEPARTURE  
1          63  DESTINATION  
2          44    DEPARTURE  
3          51  DESTINATION  
4          75    DEPARTURE  


In [11]:
# Limiter à 1000 lignes pour accélérer l'entraînement
df = df.sample(n=20000, random_state=42)
print("\ndatabase limite a 20 000 lignes pour l'entrainement rapide\n")


database limite a 20 000 lignes pour l'entrainement rapide



In [13]:
# creation d'un conteneur pour stocker les documents au format spaCy
db = DocBin()
print("\nconteneur DocBin cree avec succes\n")


conteneur DocBin cree avec succes



In [15]:
# on melange les donnees pour eviter les biais dans l'apprentissage
df = df.sample(frac=1).reset_index(drop=True)
print("\ndonnees melangees avec succes\n")


donnees melangees avec succes



In [17]:
# conversion des données au format spaCy
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    text = row['sentence']
    doc = nlp_fr.make_doc(text)  # on cree un document spaCy a partir du texte
    ents = []

    # on recupere les infos des entites
    start = row['entity_start']
    end = row['entity_end']
    label = row['entity_type']

    # on cree une entite spaCy a partir des coordonnees de debut et de fin
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is not None:
        ents.append(span)

    doc.ents = ents  # on assigne les entites au document
    db.add(doc)  # on ajoute le document au DocBin

100%|██████████| 20000/20000 [00:11<00:00, 1708.11it/s]


In [19]:
# on enregistre les donnees converties au format spaCy
db.to_disk("../../dataset/processed/processed_datas.spacy")
print("\ndonnees converties et sauvegardees au format spaCy\n")


donnees converties et sauvegardees au format spaCy



In [21]:
# ajout du composant NER au modele actuel
ner = nlp_fr.get_pipe("ner")

In [23]:
# Ajouter de nouveaux labels d'entités à partir des données
for _, row in df.iterrows():
    ner.add_label(row['entity_type'])
print("\nLes nouvelles étiquettes NER ont été ajoutées au modèle\n")


Les nouvelles étiquettes NER ont été ajoutées au modèle



In [25]:
# on cree le fichier de configuration
os.system('python -m spacy init config ./configuration.cfg --lang fr --pipeline ner --optimize efficiency --force')
print("\nfichier de configuration genere avec succes.\n")


fichier de configuration genere avec succes.



In [27]:
# entrainement avec un modele leger et config optimisee
command = 'python -m spacy train ./configuration.cfg --output ../../models/saved_models --paths.train ../../dataset/processed/processed_datas.spacy --paths.dev ../../dataset/processed/processed_datas.spacy'

with tqdm(total=100, desc="Entraînement du modèle") as pbar:
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in process.stdout:
        print(line.decode(), end='')
        pbar.update(1)

process.wait()
print("\nentraenement du modele termine avec succes\n")

Entraînement du modèle:   1%|          | 1/100 [00:19<32:25, 19.66s/it]

[i] Saving to output directory: ..\..\models\saved_models
[i] Using CPU

=========================== Initializing pipeline ===========================


Entraînement du modèle:   4%|▍         | 4/100 [00:20<08:06,  5.07s/it]


entraenement du modele termine avec succes



In [31]:
# on charge le modele entraîne
nlp_itineraire = spacy.load("../../models/saved_models/model-best")
print("modele entraine charge avec succes")

modele entraine charge avec succes


In [33]:
# Tester le modèle avec un texte plus complexe
texte_test = '''
Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence.
Je compte me rendre à Bordeaux depuis Marseille pour rendre visite à ma soeur.
Je dois regarder les trains Toulouse - Brest pour aller voir mon ami Albert.
Je dois planifier un voyage Nice Toulouse pour les prochaines vacances.
Une réunion de travail m'oblige à faire Paris - Clermont-Ferrand la semaine prochaine.
'''

# Utiliser le modèle pour identifier les entités dans le texte
doc_test = nlp_itineraire(texte_test)

# Afficher les entités reconnues
spacy.displacy.render(doc_test, style="ent", jupyter=True)